In [1]:
# Read Images 

from SimpleITK import ImageSeriesReader
import sys

input_data_directory = "./../sample-dicom/input"

series_IDs = ImageSeriesReader.GetGDCMSeriesIDs(input_data_directory)
if not series_IDs:
    print(
        'ERROR: given directory "'
        + input_data_directory
        + '" does not contain a DICOM series.'
    )
    sys.exit(1)

series_file_names = ImageSeriesReader.GetGDCMSeriesFileNames(
    input_data_directory, series_IDs[0]
)

series_reader = ImageSeriesReader()
series_reader.SetFileNames(series_file_names)
series_reader.MetaDataDictionaryArrayUpdateOn()
series_reader.LoadPrivateTagsOn()
image = series_reader.Execute()

In [2]:
# Reorient the image to obtain the sagittal view

from SimpleITK import PermuteAxes, Flip

sagittal_image = PermuteAxes(image, [2, 0, 1])
sagittal_image = Flip(sagittal_image, [False, False, True])

coronal_image = PermuteAxes(image, [0, 2, 1])
coronal_image = Flip(coronal_image, [False, False, True])

axial_image = image


print("axial_image", axial_image.GetWidth(), axial_image.GetHeight(),  axial_image.GetDepth())
print("sagittal_image", sagittal_image.GetWidth(), sagittal_image.GetHeight(),  sagittal_image.GetDepth())
print("coronal_image", coronal_image.GetWidth(), coronal_image.GetHeight(),  coronal_image.GetDepth())


axial_image 561 561 401
sagittal_image 401 561 561
coronal_image 561 401 561


In [3]:
# Process image to add adaptive histogram, to make it look better

useAdaptiveHistogram = True

from SimpleITK import AdaptiveHistogramEqualization, GetArrayViewFromImage
from ipywidgets import IntProgress
from IPython.display import display

import matplotlib.pyplot as plt


input_image = sagittal_image

print("sagittal", input_image.GetWidth(), input_image.GetHeight(),  input_image.GetDepth())

result_slices = []
progress = IntProgress(min=0, max=image.GetDepth())
display(progress)

for i in range(0, input_image.GetDepth()):
    progress.value = i
    slice = input_image[:, :, i]
    if useAdaptiveHistogram:
        slice = AdaptiveHistogramEqualization(slice, slice.GetSize())
    result_slices.append(slice)

print("Width:", input_image.GetWidth() , ", Height:", input_image.GetHeight() , ", Depth: " , input_image.GetDepth())
print("Slices Count:", len(result_slices), ", Slide Width:", result_slices[0].GetWidth(), ", Slide Height:",result_slices[0].GetHeight())

plt.figure(figsize=(10, 10))
plt.imshow(GetArrayViewFromImage(result_slices[len(result_slices)//2]), cmap="gray")
plt.plot()


sagittal 401 561 561


IntProgress(value=0, max=401)

KeyboardInterrupt: 

In [ ]:
# View changed images 

from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
from SimpleITK import GetArrayViewFromImage

slices = result_slices

def plot(slice_num): 
    slice = slices[slice_num]
    output_slice_array = GetArrayViewFromImage(slice)
    plt.figure(figsize=(10, 10))
    plt.imshow(output_slice_array, cmap="gray")
    plt.plot()


slider = widgets.IntSlider(
    value=input_image.GetSize()[2]//2,
    min=0,
    max=input_image.GetSize()[2]-1,
    step=1,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

print({"Slide Width": slices[0].GetWidth(), "Slide Height": slices[0].GetHeight()})
interact(plot, slice_num=slider)


{'Slide Width': 401, 'Slide Height': 561}


interactive(children=(IntSlider(value=280, description='slice_num', max=560), Output()), _dom_classes=('widget…

<function __main__.plot(slice_num)>

In [ ]:
# Write Slices to file

from SimpleITK import ImageSeriesWriter, Cast, RescaleIntensity, sitkUInt8, WriteImage
import os
from ipywidgets import IntProgress


type = "sagittal"
output_folder = "./../sample-dicom/output/" + type

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

writer = ImageSeriesWriter()
slices = result_slices

progress = IntProgress(min=0, max=len(slices))
display(progress)

# Iterate over each slice and save as PNG
for i in range(len(slices)):
    # Extract the slice
    progress.value = i
    slice_image = slices[i]

    # Convert the slice to a format suitable for PNG
    slice_image = RescaleIntensity(slice_image, 0, 255)  # Rescale to 0-255
    slice_image = Cast(slice_image, sitkUInt8)

    # Define the filename for the PNG
    png_filename = os.path.join(output_folder, f'slice_{i:03d}.png')

    # Write the PNG image
    WriteImage(slice_image, png_filename)



In [ ]:
# Create 3D GLTF
import numpy as np
import trimesh
import open3d as o3d

from SimpleITK import GetArrayFromImage
from skimage import measure


# Step 2: Convert the SimpleITK image to a NumPy array
array = GetArrayFromImage(image)

# Step 3: Apply a threshold to segment the volume
# Adjust the lower/upper threshold based on the structure of interest
lower_threshold = 500  # Soft tissue threshold for CT
upper_threshold = 1000  # Bone density
binary_mask = np.logical_and(array > lower_threshold, array < upper_threshold)


# Step 4: Extract a 3D mesh using the Marching Cubes algorithm
verts, faces, _, _ = measure.marching_cubes(binary_mask, level=0.5)

# Step 5: Create a mesh object using trimesh
mesh = trimesh.Trimesh(vertices=verts, faces=faces)

# Apply Laplacian Smoothing to reduce noise
mesh = mesh.filter_smooth_laplacian(number_of_iterations=10)
# Optionally, apply Taubin smoothing (reduces volume shrinkage)
mesh = mesh.filter_smooth_taubin(number_of_iterations=10)

# Save the denoised model
o3d.io.write_triangle_mesh("output_model_denoised.gltf", mesh)

print("GLTF model saved as output_model.gltf")


GLTF model saved as output_model.gltf


# Notes

In [ ]:
def get_series_tags(direction):
    
    # Write the 3D image as a series
    # IMPORTANT: There are many DICOM tags that need to be updated when you modify
    #            an original image. This is a delicate operation and requires
    #            knowledge of the DICOM standard. This example only modifies some.
    #            For a more complete list of tags that need to be modified see:
    #                http://gdcm.sourceforge.net/wiki/index.php/Writing_DICOM

    # Copy relevant tags from the original meta-data dictionary (private tags are
    # also accessible).
    tags_to_copy = [
        "0010|0010",  # Patient Name
        "0010|0020",  # Patient ID
        "0010|0030",  # Patient Birth Date
        "0020|000d",  # Study Instance UID, for machine consumption
        "0020|0010",  # Study ID, for human consumption
        "0008|0020",  # Study Date
        "0008|0030",  # Study Time
        "0008|0050",  # Accession Number
        "0008|0060",  # Modality
    ]

    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d")

    # Copy some of the tags and add the relevant tags indicating the change.
    # For the series instance UID (0020|000e), each of the components is a number,
    # cannot start with zero, and separated by a '.' We create a unique series ID
    # using the date and time.
    # NOTE: Always represent DICOM tags using lower case hexadecimals.
    #       DICOM tags represent hexadecimal numbers, so 0020|000D and 0020|000d
    #       are equivalent. The ITK/SimpleITK dictionary is string based, so these
    #       are two different keys, case sensitive. When read from a DICOM file the
    #       hexadecimal string representations are in lower case. To ensure consistency,
    #       always use lower case for the tags.
    # Tags of interest:
    series_tag_values = [
        (k, series_reader.GetMetaData(0, k))
        for k in tags_to_copy
        if series_reader.HasMetaDataKey(0, k)
    ] + [
        ("0008|0031", modification_time),  # Series Time
        ("0008|0021", modification_date),  # Series Date
        ("0008|0008", "DERIVED\\SECONDARY"),  # Image Type
        (
            "0020|000e",
            "1.2.826.0.1.3680043.2.1125." + modification_date + ".1" + modification_time,
        ),
        # Series Instance UID
        (
            "0020|0037",
            "\\".join(
                map(
                    str,
                    (
                        direction[0],
                        direction[3],
                        direction[6],
                        direction[1],
                        direction[4],
                        direction[7],
                    ),  # Image Orientation (Patient)
                )
            ),
        ),
        (
            "0008|103e",
            (
                series_reader.GetMetaData(0, "0008|103e")
                if series_reader.HasMetaDataKey(0, "0008|103e")
                else "" + " Processed-SimpleITK"
            ),
        ),  # Series Description is an optional tag, so may not exist
    ]

    return series_tag_values


get_series_tags(sagittal_image.GetDirection())


In [ ]:
output_data_directory = "./../sample-dicom/output"

def writeSlices()
    writer = sitk.ImageFileWriter()
    # Use the study/series/frame of reference information given in the meta-data
    # dictionary and not the automatically generated information from the file IO
    writer.KeepOriginalImageUIDOn()

    

    for i in range(imageToWrite.GetDepth()):
        image_slice = filtered_image[:, :, i]
        # Tags shared by the series.
        for tag, value in series_tag_values:
            image_slice.SetMetaData(tag, value)
        # Slice specific tags.
        #   Instance Creation Date
        image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d"))
        #   Instance Creation Time
        image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S"))
        #   Image Position (Patient)
        image_slice.SetMetaData(
            "0020|0032",
            "\\".join(map(str, filtered_image.TransformIndexToPhysicalPoint((0, i, 0)))),
        )
        #   Instance Number
        image_slice.SetMetaData("0020|0013", str(i))

        # Write to the output directory and add the extension dcm, to force writing
        # in DICOM format.
        writer.SetFileName(os.path.join(output_data_directory, str(i) + ".dcm"))
        writer.Execute(image_slice)

In [ ]:
from SimpleITK import AdaptiveHistogramEqualization, GetArrayViewFromImage
import matplotlib.pyplot as plt

def show(slice, name):
     slice = AdaptiveHistogramEqualization(slice, slice.GetSize())
     output_slice_array = GetArrayViewFromImage(slice)
     plt.figure(figsize=(8, 8))
     plt.imshow(output_slice_array, cmap="gray")
     plt.title(name)
     plt.axis("off")
     plt.show()

In [ ]:
# Set Image Viewer to see Dicom files
image_viewer = sitk.ImageViewer()
image_viewer.SetTitle('grid using ImageViewer class')
image_viewer.SetApplication('/Applications/ITK-SNAP.app/Contents/MacOS/ITK-SNAP')

In [ ]:
output = sitk.Flip(image3D, [False, True, False])

print(output.GetDirection())
print(output.GetSize())
print(output.GetOrigin())
print(output.GetSpacing())
plt.imshow(output[:, :, 100])

In [ ]:
# This is incorrect

# Resample to perform MPR (e.g., to get a sagittal plane)

image = image3D
resampler = sitk.ResampleImageFilter()
resampler.SetOutputDirection(image.GetDirection())
resampler.SetSize([image.GetSize()[2], image.GetSize()[1], image.GetSize()[0]])  # Change axes for sagittal plane
resampler.SetOutputOrigin(image.GetOrigin())
resampler.SetOutputSpacing([image.GetSpacing()[2], image.GetSpacing()[1], image.GetSpacing()[0]])
filtered_image = resampler.Execute(sitk.Flip(image3D, [False, True, False]))

print(image3D.GetDirection())
print(filtered_image.GetDirection())
